# Multiple Modelos de ANN
## Generar multiples modelos a partir de un grafico de capas.
### Link : https://www.tensorflow.org/guide/keras/functional

## Actualizar a Python 3.10

In [1]:
!sudo apt-get install python3.10
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!sudo update-alternatives --set python3 /usr/bin/python3.10
!sudo update-alternatives --config python3
!sudo apt install python3-pip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libpython3.10-minimal libpython3.10-stdlib python3.10-minimal
Suggested packages:
  python3.10-venv binfmt-support
The following NEW packages will be installed:
  libpython3.10-minimal libpython3.10-stdlib python3.10 python3.10-minimal
0 upgraded, 4 newly installed, 0 to remove and 20 not upgraded.
Need to get 5,097 kB of archives.
After this operation, 19.5 MB of additional disk space will be used.
Get:1 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic/main amd64 libpython3.10-minimal amd64 3.10.7-1+bionic1 [823 kB]
Get:2 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic/main amd64 python3.10-minimal amd64 3.10.7-1+bionic1 [1,970 kB]
Get:3 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bion

In [2]:
!python3 --version

Python 3.10.7


## Librerias

In [21]:
import tensorflow as tf
tf.keras.backend.clear_session()

## General multiples modelos a partir de un unico grafico de capas.
### Convolutional Neural NEtwork CNN
El reverso de una capa Conv2D es una capa Conv2DTranspose, y el reverso de una capa MaxPooling2D es una capa UpSampling2D.

Los autoencoders son un tipo de redes neuronales en las que la entrada y salida del modelo es la misma, es decir, redes entrenadas para predecir un resultado igual a los datos de entrada.

In [4]:
encoder_inputs = tf.keras.Input(shape=(28, 28, 1), name='imgs')
x = tf.keras.layers.Conv2D(16, 3, activation='relu')(encoder_inputs)
x = tf.keras.layers.Conv2D(32, 3, activation='relu')(x)
x = tf.keras.layers.MaxPooling2D(3)(x)
x = tf.keras.layers.Conv2D(32, 3, activation='relu')(x)
x = tf.keras.layers.Conv2D(16, 3, activation='relu')(x)
encoder_outputs = tf.keras.layers.GlobalMaxPooling2D()(x)

encoder_model = tf.keras.Model(inputs=encoder_inputs, outputs=encoder_outputs, name='Model_Encoder')
encoder_model.summary()

Model: "Model_Encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 imgs (InputLayer)           [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 16)        160       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        4640      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 8, 8, 32)         0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 6, 6, 32)          9248      
                                                                 
 conv2d_3 (Conv2D)           (None, 4, 4, 16)          4624      
                                                     

In [5]:
x = tf.keras.layers.Reshape((4, 4, 1))(encoder_outputs)
x = tf.keras.layers.Conv2DTranspose(16, 3, activation='relu')(x)
x = tf.keras.layers.Conv2DTranspose(32, 3, activation='relu')(x)
x = tf.keras.layers.UpSampling2D(3)(x)
x = tf.keras.layers.Conv2DTranspose(16, 3, activation='relu')(x)
decoder_outputs = tf.keras.layers.Conv2DTranspose(1, 3, activation='relu')(x)

autoencoder_model = tf.keras.Model(inputs=encoder_inputs, outputs=decoder_outputs, name='Model_Autoencoder')
autoencoder_model.summary()

Model: "Model_Autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 imgs (InputLayer)           [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 16)        160       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        4640      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 8, 8, 32)         0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 6, 6, 32)          9248      
                                                                 
 conv2d_3 (Conv2D)           (None, 4, 4, 16)          4624      
                                                 

## Crear un modelo a partir de multiples modelos.
### Encoder Model

In [10]:
encoder_input = tf.keras.Input(shape=(28, 28, 1), name='original_img')
x = tf.keras.layers.Conv2D(16, 3, activation='relu')(encoder_input)
x = tf.keras.layers.Conv2D(32, 3, activation='relu')(x)
x = tf.keras.layers.MaxPooling2D(3)(x)
x = tf.keras.layers.Conv2D(32, 3, activation='relu')(x)
x = tf.keras.layers.Conv2D(16, 3, activation='relu')(x)
encoder_output = tf.keras.layers.GlobalMaxPooling2D()(x)

encoder_model = tf.keras.Model(encoder_input, encoder_output, name='encoder_model')
encoder_model.summary()

Model: "encoder_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 original_img (InputLayer)   [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 16)        160       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        4640      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 8, 8, 32)         0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 6, 6, 32)          9248      
                                                                 
 conv2d_3 (Conv2D)           (None, 4, 4, 16)          4624      
                                                     

### Decoder Model

In [11]:
decoder_input = tf.keras.Input(shape=(16,), name='encoded_img')
x = tf.keras.layers.Reshape((4, 4, 1))(decoder_input)
x = tf.keras.layers.Conv2DTranspose(16, 3, activation='relu')(x)
x = tf.keras.layers.Conv2DTranspose(32, 3, activation='relu')(x)
x = tf.keras.layers.UpSampling2D(3)(x)
x = tf.keras.layers.Conv2DTranspose(16, 3, activation='relu')(x)
decoder_output = tf.keras.layers.Conv2DTranspose(1, 3, activation='relu')(x)

decoder_model = tf.keras.Model(decoder_input, decoder_output, name='decoder_model')
decoder_model.summary()

Model: "decoder_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoded_img (InputLayer)    [(None, 16)]              0         
                                                                 
 reshape (Reshape)           (None, 4, 4, 1)           0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 6, 6, 16)         160       
 nspose)                                                         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 8, 8, 32)         4640      
 ranspose)                                                       
                                                                 
 up_sampling2d (UpSampling2D  (None, 24, 24, 32)       0         
 )                                                               
                                                     

### Autoencoder Model
Crear un nuevo Modelo a partir de 2 Modelo existentes llamandolos como Capas.

In [15]:
# Re defino la Input Layer
autoencoder_inputs = tf.keras.Input(shape=(28, 28, 1), name='img')
# Creo el primer grafico de capas a partir de un modelo existente
encoded_img = encoder_model(autoencoder_inputs)
# Creo el segundo grafico de capas pasando el primer grafico como capa.
decoded_img = decoder_model(encoded_img)
# Creo el modelo a partir de los dos graficos de capas anteriores.
autoencoder_model = tf.keras.Model(inputs=autoencoder_inputs, outputs=decoded_img, name='autoencoder_model')
autoencoder_model.summary()

Model: "autoencoder_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 img (InputLayer)            [(None, 28, 28, 1)]       0         
                                                                 
 encoder_model (Functional)  (None, 16)                18672     
                                                                 
 decoder_model (Functional)  (None, 28, 28, 1)         9569      
                                                                 
Total params: 28,241
Trainable params: 28,241
Non-trainable params: 0
_________________________________________________________________


## Anidar/Ensamblar 3 Modelos.

In [22]:
def get_model(inputs):
  # inputs = tf.keras.Input(shape=(128,))
  outputs = tf.keras.layers.Dense(units=10, activation='sigmoid')(inputs)
  return tf.keras.Model(inputs=inputs, outputs=outputs)

inputs = tf.keras.Input(shape=(128, ))
m1 = get_model(inputs)
m2 = get_model(inputs)
m3 = get_model(inputs)

model_1 = m1(inputs)
model_2 = m1(inputs)
model_3 = m1(inputs)
outputs = tf.keras.layers.average([model_1, model_2, model_3])

ensemble_model = tf.keras.Model(inputs=inputs, outputs=outputs)
ensemble_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 model (Functional)             (None, 10)           1290        ['input_1[0][0]',                
                                                                  'input_1[0][0]',                
                                                                  'input_1[0][0]']                
                                                                                                  
 average (Average)              (None, 10)           0           ['model[0][0]',                  
                                                                  'model[1][0]',            